In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import os
import cv2
import numpy as np

In [2]:
num_gpus = len(tf.config.list_physical_devices('GPU'))
print(num_gpus)

1


2025-03-01 22:19:03.754576: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-01 22:19:03.865110: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-01 22:19:03.865449: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
# Hyper-parameters
resized_width, resized_hieght = 64, 64


In [6]:
# Load Dataset
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            # Resize to a fixed size
            img = cv2.resize(img, (resized_width, resized_hieght))  
            images.append(img)
            labels.append(label)
    return images, labels

# Define Paths
mask_folder = "../data/with_mask_without_mask_classification/with_mask"
no_mask_folder = "../data/with_mask_without_mask_classification/without_mask"

# Load Images
mask_images, mask_labels = load_images_from_folder(mask_folder, 1)
no_mask_images, no_mask_labels = load_images_from_folder(no_mask_folder, 0)

In [7]:
# Combine Data
X = np.array(mask_images + no_mask_images)
y = np.array(mask_labels + no_mask_labels)

# Convert to grayscale for feature extraction
X_gray = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in X])

In [8]:
# Normalize Images
X_norm = X.astype("float32") / 255.0  # Scale pixel values to [0, 1]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42)


In [9]:
# Normalize Images
X_norm = X.astype("float32") / 255.0  # Scale pixel values to [0, 1]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42)

# CNN Model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train Model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"CNN Accuracy: {test_acc:.2f}")


2025-03-01 22:49:45.019294: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-01 22:49:45.019646: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-01 22:49:45.019891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/10


2025-03-01 22:49:46.442193: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8907
2025-03-01 22:49:46.830615: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-03-01 22:49:46.907114: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f4cd0e7acc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-03-01 22:49:46.907198: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-03-01 22:49:46.930736: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-03-01 22:49:47.144662: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

35/35 [==============================] - 3s 18ms/step - loss: 0.4442 - accuracy: 0.8015 - val_loss: 0.2497 - val_accuracy: 0.9094
Epoch 2/10
35/35 [==============================] - 0s 5ms/step - loss: 0.2067 - accuracy: 0.9229 - val_loss: 0.3477 - val_accuracy: 0.8804
Epoch 3/10
35/35 [==============================] - 0s 6ms/step - loss: 0.2116 - accuracy: 0.9284 - val_loss: 0.1587 - val_accuracy: 0.9384
Epoch 4/10
35/35 [==============================] - 0s 6ms/step - loss: 0.1393 - accuracy: 0.9565 - val_loss: 0.1414 - val_accuracy: 0.9601
Epoch 5/10
35/35 [==============================] - 0s 6ms/step - loss: 0.1545 - accuracy: 0.9538 - val_loss: 0.1349 - val_accuracy: 0.9529
Epoch 6/10
35/35 [==============================] - 0s 6ms/step - loss: 0.1272 - accuracy: 0.9619 - val_loss: 0.0823 - val_accuracy: 0.9819
Epoch 7/10
35/35 [==============================] - 0s 6ms/step - loss: 0.1111 - accuracy: 0.9646 - val_loss: 0.0747 - val_accuracy: 0.9746
Epoch 8/10
35/35 [============